This code snippet is about a project, aiming towards predicting the unloading time at a customer location. Main objective was to find a regression formula, whether linear or polynomial, in order to support optimization algorithm with a better parameter estimation. We used to use a fixed unloading time before this proect. We decided to investigate if there may be a dynamic alternative to this fixed parameter. Unfortunately, data collection in this part of our company is a little bit dirty and without a clear procedure. This led to bad performance for regression algorithm in the end. 

In [20]:
import pandas as pd
import re
import numpy as np
pd.set_option('display.max_columns',40)
np.set_printoptions(threshold=np.inf)
import more_itertools as mi
import timeit
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import warnings
import random
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import anderson, normaltest
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning

In [21]:
random.seed(230)
np.random.seed(230)
pd.options.mode.chained_assignment = None

text_file2 = open(r'C:\Users\ali.kilinc\Desktop\Nike Süre Tahminleme\FinalizeOut.txt',"w")

warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

start_time = timeit.default_timer()

data = pd.read_excel(r'C:\Users\ali.kilinc\Desktop\Nike Süre Tahminleme\Nike_Input.xlsx', index_col= False)

Below code is for initial filtering of unrealistic values, filtering non-rare unloading locations, and some drop of unnecessary columns. 

In [22]:
data = data[data['UNLOAD_TIME'] < 3600]
data = data[data['TIME_PER_DESI'] < 24]
data = data[data['TIME_PER_DESI'] > 0.5]
data = data[data['TIME_PER_KOLI'] < 600]
data = data[data['TIME_PER_KOLI'] > 12.5]

#data = data[~data['VH_TYPE'].isin(['KAMYON', 'TIR'])]

data['DATE_ARRIVAL'] = pd.to_datetime(data['DATE_ARRIVAL'], format = '%d/%m/%Y %H:%M:%S')
data['YEAR'] = data['DATE_ARRIVAL'].dt.year.astype(str)
data['MONTH'] = data['DATE_ARRIVAL'].dt.month.astype(str)

data = data[data['YEAR'] >= '2019']
#data = data[data['MONTH'] >= '7']

data = data[~data['UNL_CC'].isin(list(data['UNL_CC'][data.groupby('UNL_CC')['UNL_CC'].transform('size') < 20]))]

dropped = data[['POSITION_ID', 'DATE_ARRIVAL', 'YEAR', 'MONTH', 'DATE_UNLOADING_END', 'NAME', 'TIME_PER_DESI', 'TIME_PER_KOLI', 
                 'ARRIVAL_MONTH', 'ARRIVAL_QUARTER', 'UNL_CC', 'UNLOADING_CITY', 'VH_TYPE','ARRIVAL_DAY']]
data.drop(columns = ['POSITION_ID', 'DATE_ARRIVAL', 'YEAR', 'MONTH', 'DATE_UNLOADING_END', 'NAME', 'TIME_PER_DESI', 'TIME_PER_KOLI', 
                      'ARRIVAL_MONTH', 'ARRIVAL_QUARTER', 'UNL_CC', 'UNLOADING_CITY', 'VH_TYPE', 'ARRIVAL_DAY'], inplace = True)
print(data.describe(include='all'))
print(data.columns)
print(data.info())

               DESI        KOLI   AVM  UNLOAD_TIME  ARRIVAL_HOUR
count   5748.000000  5748.00000  5748  5748.000000   5748.000000
unique          NaN         NaN     2          NaN           NaN
top             NaN         NaN   AVM          NaN           NaN
freq            NaN         NaN  3605          NaN           NaN
mean     403.678873    16.41110   NaN   730.981907     10.924322
std      636.714177    24.07784   NaN   847.033316      2.108179
min        3.000000     1.00000   NaN    60.000000      2.000000
25%       54.000000     3.00000   NaN    89.000000      9.000000
50%      144.998500     6.00000   NaN   348.000000     10.000000
75%      465.002500    20.00000   NaN  1183.000000     12.000000
max     6708.000000   258.00000   NaN  3598.000000     23.000000
Index(['DESI', 'KOLI', 'AVM', 'UNLOAD_TIME', 'ARRIVAL_HOUR'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5748 entries, 14693 to 24686
Data columns (total 5 columns):
DESI            5748 non-null f

In [24]:
#data['VH_TYPE'] = data['VH_TYPE'].astype('category')
#data['UNLOADING_CITY'] = data['UNLOADING_CITY'].astype('category')
data['ARRIVAL_HOUR'] = data['ARRIVAL_HOUR'].astype('category')
data['AVM'] = data['AVM'].astype('category')
#data['UNL_CC'] = data['UNL_CC'].astype('category')
print(data.info())

transform = 1
regress = 1
finalize = 1

#data['UNL_CC'].replace(sorted(set(data['UNL_CC'][data.groupby('UNL_CC')['UNL_CC'].transform('size') < 150])) , 'othcc', inplace = True)
#data['UNL_CC'] = data['UNL_CC'].astype('str')

#print(data['VH_TYPE'].value_counts())
#print(data['UNLOADING_CITY'].value_counts())
print(data['ARRIVAL_HOUR'].value_counts())
#print(data['ARRIVAL_DAY'].value_counts())
#print(data['ARRIVAL_MONTH'].value_counts())
#print(data['ARRIVAL_QUARTER'].value_counts())
print(data['AVM'].value_counts())
#print(data['UNL_CC'].value_counts())

#data = data[data['UNL_CC'] != 'othcc']

threshold = 125

print(sorted(set(data['ARRIVAL_HOUR'][data.groupby('ARRIVAL_HOUR')['ARRIVAL_HOUR'].transform('size') < threshold])))

data['ARRIVAL_HOUR'].replace(sorted(set(data['ARRIVAL_HOUR'][data.groupby('ARRIVAL_HOUR')['ARRIVAL_HOUR'].transform('size') < threshold])) , 'oth', inplace = True)
data['ARRIVAL_HOUR'] = data['ARRIVAL_HOUR'].astype('str')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5748 entries, 14693 to 24686
Data columns (total 5 columns):
DESI            5748 non-null float64
KOLI            5748 non-null int64
AVM             5748 non-null category
UNLOAD_TIME     5748 non-null int64
ARRIVAL_HOUR    5748 non-null category
dtypes: category(2), float64(1), int64(2)
memory usage: 191.3 KB
None
9      1438
10     1153
11     1069
12      565
13      429
14      361
8       270
15      199
16      168
oth      96
Name: ARRIVAL_HOUR, dtype: int64
AVM     3605
NAVM    2143
Name: AVM, dtype: int64
['oth']


Above code tries to filter rare unloading hours from data, mostly the ones out-of-office-hours. 

In [25]:
"""
#!!!!!!!! IT IS THE TRANSFORMATION OF CYCLICAL VARIABLES LIKE MONTH AND QUARTER. ONE-HOT IS AN OPTION BUT IT IS MUCH MORE BETTER THAN ENCODING
df_x['month_x'] = np.sin(2*np.pi*df_x['ARRIVAL_MONTH']/12)
df_x['month_y'] = np.cos(2*np.pi*df_x['ARRIVAL_MONTH']/12)

df_x['quarter_x'] = np.sin(2*np.pi*df_x['ARRIVAL_QUARTER']/4)
df_x['quarter_y'] = np.sin(2-np.pi*df_x['ARRIVAL_QUARTER]/4)

df_x['day_x'] = np.sin(2*np.pi*df_x['ARRIVAL_DAY']/7)
df_x['day_y'] = np.cos(2*np.pi*df_x['ARRIVAL_DAY']/7)

df_x.drop(columns = ['ARRIVAL_MONTH', 'ARRIVAL_QUARTER', 'ARRIVAL_DAY'], axis = 1, inplace = True)
"""
""" # General function for this transformation
def encode(data, col, max_val): # max val is theoretical maximum of column, for month it is 12, for day of year it is 365 etc.
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data
"""

#data['ARRIVAL_DAY'] = data['ARRIVAL_DAY'].astype('str')
#data['ARRIVAL_MONTH'] = data['ARRIVAL_MONTH'].astype('str')
#data['ARRIVAL_QUARTER'] = data['ARRIVAL_QUARTER'].astype('str')

df_y = data['UNLOAD_TIME']
df_x = data.drop(columns = ['UNLOAD_TIME'], axis=1)

fix_cols = []
for col in df_x.columns:
    if df_x[col].nunique() == 1:
        fix_cols.append(col)
    else:
        pass

emp_cols = []
for col in df_x.columns:
    if df_x[col].count() > 0:
        pass
    else:
        emp_cols.append(col)

df_x.drop(columns = list(emp_cols+fix_cols), inplace = True)

df_x.drop(columns = ['KOLI'], inplace = True)
num_cols = list(df_x._get_numeric_data().columns)

print(df_x.info())

cat_cols = list(set(df_x.columns) - set(num_cols) - set(emp_cols) - set(fix_cols))
num_cols = list(set(num_cols) - set(fix_cols) - set(emp_cols))

print(cat_cols)
print(num_cols)

df_x = pd.DataFrame(df_x, columns = df_x.columns)
df_y = pd.DataFrame(df_y, columns = ['UNLOAD_TIME'])

"""
sns.pairplot(df_x)

vif = [variance_inflation_factor(df_x[num_cols].values, i) for i in range(df_x[num_cols].shape[1])]
print(vif)

sns.heatmap(df_x[num_cols].corr(), annot = True)
plt.show()

sns.distplot(df_y.values.ravel(),color="g")
"""

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5748 entries, 14693 to 24686
Data columns (total 3 columns):
DESI            5748 non-null float64
AVM             5748 non-null category
ARRIVAL_HOUR    5748 non-null object
dtypes: category(1), float64(1), object(1)
memory usage: 140.4+ KB
None
['AVM', 'ARRIVAL_HOUR']
['DESI']


'\nsns.pairplot(df_x)\n\nvif = [variance_inflation_factor(df_x[num_cols].values, i) for i in range(df_x[num_cols].shape[1])]\nprint(vif)\n\nsns.heatmap(df_x[num_cols].corr(), annot = True)\nplt.show()\n\nsns.distplot(df_y.values.ravel(),color="g")\n'

In [26]:
#data shuffler
idx = np.random.permutation(df_x.index)
df_x = df_x.reindex(idx)
df_y = df_y.reindex(idx)

print(df_x.info())
print(df_y.info())

#print(df_x['ARRIVAL_HOUR'].value_counts())

#y is a lognormal distribution with mostly 0 values, i will try to fit a log(x+1) transformation for that.( same for all month variables)
#log1p(x) is equal to log(x+1). reverse of it is expm1(y)

"""
#for normality test
result = anderson(df_y.values.ravel())
print('Statistic: %.3f' % result.statistic)
p = 0
for i in range(len(result.critical_values)):
	sl, cv = result.significance_level[i], result.critical_values[i]
	if result.statistic < result.critical_values[i]:
		print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
	else:
		print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))

stat, p1 = normaltest(df_y.values.ravel())
print('Statistics=%.3f, p1=%.3f' % (stat, p1))
# interpret
alpha = 0.05
if p > alpha:
	print('Sample looks Gaussian (fail to reject H0)')
else:
	print('Sample does not look Gaussian (reject H0)')
"""

if transform == 1:
    #sns.distplot(df_y, hist = False, kde = True)
    #plt.show()
    #df_y = np.log1p(df_y)
    df_y = np.log(df_y)
    #df_y = pow(df_y, 0.5)
    #df_y = np.log(df_y)
    #sns.distplot(df_y, hist = True, kde = True)
    #plt.show()
    #df_x[month_var] = np.log(df_x[month_var]+1)
    #pred_set[month_var] = np.log(pred_set[month_var]+1)
else:
    pass



<class 'pandas.core.frame.DataFrame'>
Int64Index: 5748 entries, 21302 to 22694
Data columns (total 3 columns):
DESI            5748 non-null float64
AVM             5748 non-null category
ARRIVAL_HOUR    5748 non-null object
dtypes: category(1), float64(1), object(1)
memory usage: 140.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5748 entries, 21302 to 22694
Data columns (total 1 columns):
UNLOAD_TIME    5748 non-null int64
dtypes: int64(1)
memory usage: 89.8 KB
None


Above code is for shuffling data, then there is a section for normality test. And also an optional transformation section for dependent variable if it is skewed.

In [31]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler, Normalizer, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold, cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, cohen_kappa_score
from xgboost import XGBClassifier
import sklearn.metrics as m
from lightgbm import LGBMClassifier
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression, Lasso, Ridge, ElasticNet, HuberRegressor, Lars, RANSACRegressor, PassiveAggressiveRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor

In [32]:
if regress == 1:

    models = dict()
    
    paramslr = {
                }

    paramslas = {
            'est__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10],
            'est__max_iter': np.linspace(0, 3000, 7, endpoint = True),
            'est__tol': [0.0001, 0.001, 0.01, 0.1]
                }
    
    paramsrid = {
            'est__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10],
            'est__max_iter': np.linspace(0, 3000, 7, endpoint = True),
            'est__tol': [0.0001, 0.001, 0.01, 0.1]
                }
    
    paramsela = {
            'est__alpha': [0.001, 0.01, 0.1, 1],
            'est__max_iter': np.linspace(0, 3000, 3, endpoint = True),
            'est__tol': [0.0001, 0.001, 0.01],
            'est__l1_ratio': np.linspace(0.25,0.75, 3, endpoint = True)
            }
    
    paramslgb = {
            'est__max_depth':[5, 9, 18, 32],
            'est__learning_rate': [0.001, 0.01, 0.1],
            'est__n_estimators': [10, 50, 100, 200],
            'est__num_leaves': np.linspace(11,51,3,endpoint = True, dtype = int)
            }
    
    models['lr'] = [LinearRegression(), paramslr]
    models['las'] = [Lasso(), paramslas]
    models['rid'] = [Ridge(), paramsrid]
    #models['ela'] = [ElasticNet(), paramsela]
    #models['lgb'] = [LGBMRegressor(), paramslgb]
    
    for i in cat_cols:
        print(list(mi.unique_everseen(df_x[i])))
    
    for key, value in models.items():
        
        #this part is for only gridsearchcv. We can use CV results, as well as best parameters of the grid search from here. 
    
        start_time = timeit.default_timer()
        
        scorer = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']
        
        preprocessor = ColumnTransformer(transformers = [('num', MinMaxScaler(feature_range = (0,1)), num_cols), 
                                                         ('cat', OneHotEncoder(sparse = True, handle_unknown = 'ignore'), cat_cols)])
        
        poly = PolynomialFeatures(3)
        
        all_pipe = Pipeline(steps = [('prep', preprocessor),('poly', poly), ('est', value[0])])
        
        search_space = value[1]
                
        #njobs = -1 uses 2 cores (all available) and 4 threads; consumes 100% of CPU. But it is faster absolutely. Njobs = 1 uses 1 core only, slower but consumes less CPU
        grid_search = GridSearchCV(all_pipe, search_space, cv=5, verbose=1, refit = 'neg_mean_squared_error', scoring = scorer, return_train_score = True, n_jobs = -1)
        #scoring option is for defining multiple scorers, otherwise null is OK
        #refit must be chosen if multi scorers is selected. But best_score_, best_params_ etc will give only the result of that metric, cant get multi scores
        #here REFIT option tells you which metric do you want to consider for best_params_ calculation(according to which metric)
        
        grid_search.fit(df_x, df_y.values.ravel())
        
        #from the resulting parameters dictionary, we choose the index of best_param_ set. This index will help us the get best param value from subresults
        #metric results are in arraf form, in each array there is a list of results corresponding to the all parameter combinations. 
        ind = grid_search.best_index_      
        
        print("model = {}".format(key))
        print("train_mse = {}, test_mse ={}".format(grid_search.cv_results_['mean_train_neg_mean_squared_error'][ind], grid_search.cv_results_['mean_test_neg_mean_squared_error'][ind]))
        print("train_mae = {}, test_mae ={}".format(grid_search.cv_results_['mean_train_neg_mean_absolute_error'][ind], grid_search.cv_results_['mean_test_neg_mean_absolute_error'][ind]))
        print("train_r2 = {}, test_r2 = {}".format(grid_search.cv_results_['mean_train_r2'][ind], grid_search.cv_results_['mean_test_r2'][ind]))
        print("avg_fit_time = {}".format(grid_search.cv_results_['mean_fit_time'][ind]))
        #we are getting best_params_ from grid_search object still, it is valid
        print("best_params = {}".format(grid_search.best_params_))
        print("---%0.1f minutes---" %((timeit.default_timer()-start_time)/60))
        print("...{}...".format(start_time))
        print("current_time = {}...".format(datetime.now()))
        print("current_time = {}...".format(datetime.now()))
        print("-----------------------------------")
        
        best_params = dict()
        best_params[key] = grid_search.best_params_
        
        best_results = dict()
        best_results[key] = [grid_search.cv_results_['mean_test_neg_mean_squared_error'][ind], grid_search.cv_results_['mean_test_neg_mean_absolute_error'][ind], grid_search.cv_results_['mean_test_r2'][ind]]


['AVM', 'NAVM']
['16', '8', '11', '10', '9', 'oth', '14', '12', '13', '15']
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


model = lr
train_mse = -0.6947036147253349, test_mse =-0.7320868529243543
train_mae = -0.6845138787357334, test_mae =-0.6959246215002357
train_r2 = 0.5882822509973737, test_r2 = 0.5656986556447459
avg_fit_time = 0.26725354194641116
best_params = {}
---0.0 minutes---
...9628.649251747634...
current_time = 2020-07-16 14:49:15.289688...
current_time = 2020-07-16 14:49:15.289688...
-----------------------------------
Fitting 5 folds for each of 168 candidates, totalling 840 fits


[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:   43.3s
[Parallel(n_jobs=-1)]: Done 239 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 680 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 840 out of 840 | elapsed:  1.8min finished


model = las
train_mse = -0.7101637807010525, test_mse =-0.7210167864105241
train_mae = -0.6949889442435507, test_mae =-0.7003904965086367
train_r2 = 0.5791188997754786, test_r2 = 0.5721936167329195
avg_fit_time = 0.27086315155029295
best_params = {'est__alpha': 0.0001, 'est__max_iter': 500.0, 'est__tol': 0.1}
---1.8 minutes---
...9629.325979690671...
current_time = 2020-07-16 14:51:05.473158...
current_time = 2020-07-16 14:51:05.473158...
-----------------------------------
Fitting 5 folds for each of 168 candidates, totalling 840 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   12.6s


model = rid
train_mse = -0.7021515376291443, test_mse =-0.7207235475026955
train_mae = -0.6892818135719423, test_mae =-0.6972499592852454
train_r2 = 0.5838695838506023, test_r2 = 0.572418503111378
avg_fit_time = 0.0953444480895996
best_params = {'est__alpha': 0.01, 'est__max_iter': 0.0, 'est__tol': 0.001}
---0.4 minutes---
...9739.501984188417...
current_time = 2020-07-16 14:51:31.177046...
current_time = 2020-07-16 14:51:31.177046...
-----------------------------------


[Parallel(n_jobs=-1)]: Done 840 out of 840 | elapsed:   25.4s finished


Below part is for finalizing the selected regression model with whole data. 

In [33]:
if finalize == 1:
    
    lr = LinearRegression()
    lasso = Lasso(alpha = 0.0001, max_iter = 500.0, tol = 0.0001)
    lgb = LGBMRegressor(learning_rate= 0.1, max_depth= 18, n_estimators= 100, num_leaves= 51)
    """
    numeric_pipe = make_pipeline(MinMaxScaler(feature_range = (0,1)))
    categoric_pipe = make_pipeline(OneHotEncoder(sparse = True, handle_unknown='ignore'))
    preprocessor = ColumnTransformer(transformers = [('num',numeric_pipe, num_cols), ('cat',categoric_pipe,cat_cols)])
    
    regr_pipe_final = make_pipeline(preprocessor, lr)
    
    regr_pipe_final.fit(df_x, df_y.values.ravel())
    
    print(regr_pipe_final.steps[0][1][1])
    print(regr_pipe_final.steps[1][1].coef_)
    print(regr_pipe_final.steps[1][1].intercept_)
    """
    
    #numeric_pipe = make_pipe('minmax', MinMaxScaler(feature_range = (0,1)))
    #categoric_pipe = Pipeline('onehot', OneHotEncoder(sparse = True, handle_unknown='ignore'))
    poly = PolynomialFeatures(3)
    preprocessor = ColumnTransformer(transformers = [('num', MinMaxScaler(feature_range = (0,1)), num_cols), 
                                                     ('cat', OneHotEncoder(sparse = True, handle_unknown='ignore'), cat_cols)])
    regr_pipe_final = Pipeline([('prep', preprocessor), ('pol', poly), ('regr',lasso)])
    
    regr_pipe_final.fit(df_x, df_y.values.ravel())
    
    # pol is coming after preprocessor in pipeline. So, it is using the output variables of preprocessor as input.
    # you can see the polynomial features are increased to 84, which are one hot encoded features mostly. 
    #So, model is automatically upgraidng itself after each step inside pipeline
    print(regr_pipe_final.named_steps['pol'].get_feature_names())
    print(regr_pipe_final.named_steps['prep'].transformers_[1][1].get_feature_names(cat_cols))
    print(regr_pipe_final.named_steps['regr'].coef_)
    print(regr_pipe_final.named_steps['regr'].intercept_)
    
    """
    print("polynomial_features = {}".format(regr_pipe_final.named_steps['pol'].get_feature_names()), file = text_file2)
    print("one_hot_features = {}".format(regr_pipe_final.named_steps['prep'].transformers_[1][1].get_feature_names(cat_cols)), file = text_file2)
    print("regr_coefficients = {}".format(regr_pipe_final.named_steps['regr'].coef_), file = text_file2)
    print("regr_intercept = {}".format(regr_pipe_final.named_steps['regr'].intercept_), file = text_file2)
    """
    
    var_dict = {}
    var_list = []
    var_list = num_cols + list(regr_pipe_final.named_steps['prep'].transformers_[1][1].get_feature_names(cat_cols))
    
    for i in range(len(var_list)):
        var_dict['x'+str(i)] = var_list[i]
        
    effect_dict = {}
    coeff_dict = {}
    
    pol_list = []
    pol_list = regr_pipe_final.named_steps['pol'].get_feature_names(var_list)
    
    for i in range(1,len(regr_pipe_final.named_steps['pol'].get_feature_names())):
        if regr_pipe_final.named_steps['regr'].coef_[i] != 0:
            #coeff_dict[str(regr_pipe_final.named_steps['pol'].get_feature_names()[i])] = [round(regr_pipe_final.named_steps['regr'].coef_[i], 4), var_list[i-1]]
            #effect_dict[var_list[i-1]] = round(regr_pipe_final.named_steps['regr'].coef_[i], 4)
            coeff_dict[str(regr_pipe_final.named_steps['pol'].get_feature_names()[i])] = [round(regr_pipe_final.named_steps['regr'].coef_[i], 4), pol_list[i]]
            effect_dict[pol_list[i]] = round(regr_pipe_final.named_steps['regr'].coef_[i], 4)
        else:
            pass
        
    effect_dict = dict(sorted(coeff_dict.items(), key=lambda x: x[1]))
    
    pred = regr_pipe_final.predict(df_x)
    df_x['real'] = np.exp(df_y)
    df_x['pred'] = np.exp(pred)
    
    #df_x['real'] = pow(df_y, 2)
    #df_x['pred'] = pow(pred, 2)
    print(df_x.head(5))

['1', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x0^2', 'x0 x1', 'x0 x2', 'x0 x3', 'x0 x4', 'x0 x5', 'x0 x6', 'x0 x7', 'x0 x8', 'x0 x9', 'x0 x10', 'x0 x11', 'x0 x12', 'x1^2', 'x1 x2', 'x1 x3', 'x1 x4', 'x1 x5', 'x1 x6', 'x1 x7', 'x1 x8', 'x1 x9', 'x1 x10', 'x1 x11', 'x1 x12', 'x2^2', 'x2 x3', 'x2 x4', 'x2 x5', 'x2 x6', 'x2 x7', 'x2 x8', 'x2 x9', 'x2 x10', 'x2 x11', 'x2 x12', 'x3^2', 'x3 x4', 'x3 x5', 'x3 x6', 'x3 x7', 'x3 x8', 'x3 x9', 'x3 x10', 'x3 x11', 'x3 x12', 'x4^2', 'x4 x5', 'x4 x6', 'x4 x7', 'x4 x8', 'x4 x9', 'x4 x10', 'x4 x11', 'x4 x12', 'x5^2', 'x5 x6', 'x5 x7', 'x5 x8', 'x5 x9', 'x5 x10', 'x5 x11', 'x5 x12', 'x6^2', 'x6 x7', 'x6 x8', 'x6 x9', 'x6 x10', 'x6 x11', 'x6 x12', 'x7^2', 'x7 x8', 'x7 x9', 'x7 x10', 'x7 x11', 'x7 x12', 'x8^2', 'x8 x9', 'x8 x10', 'x8 x11', 'x8 x12', 'x9^2', 'x9 x10', 'x9 x11', 'x9 x12', 'x10^2', 'x10 x11', 'x10 x12', 'x11^2', 'x11 x12', 'x12^2', 'x0^3', 'x0^2 x1', 'x0^2 x2', 'x0^2 x3', 'x0^2 x4', 'x0^2 x5', 'x0^2

This last part is for readability, and post analysis of variable coefficients. Also, it transforms the output into a more readable format by optimization algorithm. Also, it writes all of these into a text file as output. 

In [34]:
    #df_x['real'] = pow(df_y, 2)
    #df_x['pred'] = pow(pred, 2)
    print(df_x.head(5))
    
    dictcount = 0
    finalstr = str()
    for key, value in effect_dict.items():
        dictcount += 1
        substr = value[1].split()
        substr2 = str()
        for i in range(len(substr)):
            if '^' in substr[i]:
                substr[i] = substr[i].replace('^', ' ')
                if i == len(substr) - 1:
                    pass
                else:
                    substr[i] = str(substr[i] + ' ')
            else:
                if i == len(substr) - 1:
                    substr[i] = substr[i] + ' 1'
                else:
                    substr[i] = str(substr[i] + ' 1 ')
            substr2 = substr2 + substr[i]
        if dictcount != len(effect_dict.items()):
            finalstr = finalstr + str(value[0]) + ' ' + substr2 + ','
        else:
            finalstr = finalstr + str(value[0]) + ' ' + substr2

    finalstr = str(round(regr_pipe_final.named_steps['regr'].intercept_, 4)) + ' INTERCEPT 0,' + finalstr
    
    print("features = {}".format(var_dict), file = text_file2)
    print("------------------------------------", file = text_file2)
    print("------------------------------------", file = text_file2)
    print("regr_coefficients = {}".format(coeff_dict), file = text_file2)
    print("------------------------------------", file = text_file2)
    print("------------------------------------", file = text_file2)
    print("regr_effects = {}".format(effect_dict), file = text_file2)
    print("------------------------------------", file = text_file2)
    print("------------------------------------", file = text_file2)
    print("regr_intercept = {}".format(regr_pipe_final.named_steps['regr'].intercept_), file = text_file2)
    print("------------------------------------", file = text_file2)
    print("------------------------------------", file = text_file2)
    print("parsed_coefficients = {}".format(finalstr), file = text_file2)
    #print_excel = df_x.to_excel(r'C:\Users\ali.kilinc\Desktop\PredNikeLasso20filteredLastDA.xlsx', index = None, header = True, sheet_name = 'FullTable')
    

text_file2.close()

           DESI   AVM ARRIVAL_HOUR    real         pred
21302    78.995   AVM           16    79.0   141.951097
20956  2139.995  NAVM            8  3000.0  2329.675634
22845   424.004   AVM           11   679.0   611.841110
17510    79.000   AVM           11   130.0   231.974779
16387    34.908  NAVM           16    73.0    96.522385
